In [27]:
import pandas as pd
import re

In [2]:
df = pd.read_csv('민간일자리정보.csv')
df2 = pd.read_csv('일자리포털정보.csv')

In [ ]:
df2.head()

**일자리 서비스에서 필요한 컬럼 정리**

**--- 기업 및 고용정보 관련 ---**
* 종류: 민간 vs 공공
* 업종: 각 업종에 따른 필터링 (예: 사무, 회계, IT, 서비스, 미화...)
* 직종: 구인정보의 직종.
* 직무내용: 구체적인 업무 내용.

* 사업장명: 일자리를 제공하는 기관 또는 회사의 이름.
* 근무지역: 일자리의 위치. (정확한 위치 표시xx, 동까지만 가능)
* 급여: 급여 조건.
* 근무시간: 근무 시간대.
* 모집인원: 채용 예정인 인원.

**--- 지원자격 관련 ---**
* 연령: 채용 연령 제한.
* 성별: 채용 성별 조건.
* 경력조건: 필요한 경력 조건.

**--- 지원 ---**
* 구인상태: 구인중 vs 구인마감
* 접수마감: 지원 마감일.
* 지원방법: 지원하는 방법.
* 지원서류: 필요한 지원 서류.

**필수**
: 종류, 모집직종, 직무내용, 사업장명, 근무지역, 급여, 근무시간, 채용인원, 연령, 성별, 지원방법

**선택**
: 종류 (민간 공공 라벨링 잘못 된 것 있음. 사이트 자체가 잘 구분해놓지 않았음.), 업종 (추가해야함?)

In [7]:
print(df.columns)
print(df2.columns)

Index(['구인상태', '업체명', '근무지역', '직종', '연령', '성별', '급여', '근무시간', '채용인원', '근무내용',
       '지원방법', '지원서류'],
      dtype='object')
Index(['Unnamed: 0', '구인제목', '사업장명', '업종', '사업내용', '근로자수', '주소', '모집직종',
       '모집인원', '학력', '고용형태', '근무예정지', '직무내용', '경력조건', '필요경력', '급여', '평일 근무시간',
       '근무형태', '접수마감', '전형방법', '접수방법'],
      dtype='object')


In [9]:
# 컬럼 정리 
df.rename(columns={'업체명': '사업장명'}, inplace=True)
df['업종'] = None  # 추후 세분화하여 업데이트 가능

df['종류'] = df.apply(
    lambda row: '공공' if '공공' in str(row['직종']) or '공공' in str(row['근무내용']) else '민간', 
    axis=1
)

In [32]:
df['접수마감'] = '모집 완료 시 마감'

In [21]:
df['구인제목'] = df['근무지역'] + ' ' + df['직종'] + ' 모집'

In [ ]:
# 의미 없는 데이터 삭제 예: '근무내용'이 불충분한 데이터 제거
print(len(df))

df.dropna(subset=['근무내용'], inplace=True)  # '근무내용' 컬럼에서 NaN 값을 가진 행을 삭제
df = df[df['근무내용'] != '']  # '근무내용'이 비어 있는 행을 삭제
df = df[df.apply(lambda row: sum(row == '상세내용 참고'), axis=1) < 3]

# DataFrame 최종 결과 출력
print(len(df))

In [28]:
### 데이터 분리 - 근무시간, 경력조건 등 

# 근무형태와 근무시간 추출 함수
def extract_work_schedule(schedule):
    work_type = re.search(r'(주\d일|격일)', schedule)
    work_hours = re.search(r'(\d{2}:\d{2}~\d{2}:\d{2})', schedule)
    
    return (
        work_type.group(0) if work_type else None,
        work_hours.group(0) if work_hours else None
    )

# 새 컬럼 생성
df['근무형태'], df['근무시간'] = zip(*df['근무시간'].apply(extract_work_schedule))

df.head()

,구인상태,사업장명,근무지역,직종,연령,성별,급여,근무시간,채용인원,근무내용,지원방법,지원서류,업종,종류,구인제목,근무형태
0,구인중,비티엠써비스(주),성동구 성수동,건물 시설기사,~65세,남,월260만원,None,1명,<비티엠써비스(주)>건물 시설기사 1명*성수동 서울M타워 시설기사 업무*경력 3년 ...,*상담사 알선 02-6220-8640,NaN,None,민간,성동구 성수동 건물 시설기사 모집,None
1,구인중,위한씨엠(주),동대문구 제기동,병원 청소원,무관,여,월147만원,12:00~17:00,1명,<위한씨엠(주)>병원 청소원 1명*제기동역 인근 동대문날개병원 청소*월147만원*평...,*상담사 알선 02-6220-8640,NaN,None,민간,동대문구 제기동 병원 청소원 모집,None
2,구인중,효성렌트카 주식회사,구로구 구로동,렌트카 배차 및 회차 직원,~70세,남,월245만원,07:00~19:00,2명,"<효성렌트카 주식회사>렌트카 배차 및 회차업무 사원 2명차량운전 및 관리, 차량배차...",*상담사 알선 02-6220-8640,NaN,None,민간,구로구 구로동 렌트카 배차 및 회차 직원 모집,주6일
3,구인중,강남시니어클럽,서울지역,카드배송원,만 60세 이상(만 65세 이상 우대),무관,월50만원 이상,09:00~15:00,00명,업무내용: 도보를 이용하여 소화물(카드 등)을 배달하는 업무.* 참여조건1. 만 6...,02-547-8866 서울강남시니어클럽,NaN,None,민간,서울지역 카드배송원 모집,주5일
4,구인중,제이앤비이에스,강남구 역삼동/삼성동/논현동,가정방문 도시가스 점검원,~60세,여,월220만원,None,2명,"<제이앤비이에스>가정방문 도시가스 점검원 2명강남구 역삼동, 삼성, 논현 인근 가정...",*상담사 알선 02-6220-8640,NaN,None,민간,강남구 역삼동/삼성동/논현동 가정방문 도시가스 점검원 모집,주5일


In [33]:
# 경력 조건 추출 함수
def extract_career_condition(text):
    # "경력 필수" 또는 "경력 2년 이상" 등의 표현 찾기
    if re.search(r'경력 필수|경력 \d+년', text):
        return '경력'
    return '무관'

# '경력조건' 컬럼 생성
df['경력조건'] = df['근무내용'].apply(extract_career_condition)

# 결과 출력
df.head()

,구인상태,사업장명,근무지역,직종,연령,성별,급여,근무시간,채용인원,근무내용,지원방법,지원서류,업종,종류,구인제목,근무형태,경력조건,접수마감
0,구인중,비티엠써비스(주),성동구 성수동,건물 시설기사,~65세,남,월260만원,None,1명,<비티엠써비스(주)>건물 시설기사 1명*성수동 서울M타워 시설기사 업무*경력 3년 ...,*상담사 알선 02-6220-8640,NaN,None,민간,성동구 성수동 건물 시설기사 모집,None,경력,모집 완료 시 마감
1,구인중,위한씨엠(주),동대문구 제기동,병원 청소원,무관,여,월147만원,12:00~17:00,1명,<위한씨엠(주)>병원 청소원 1명*제기동역 인근 동대문날개병원 청소*월147만원*평...,*상담사 알선 02-6220-8640,NaN,None,민간,동대문구 제기동 병원 청소원 모집,None,무관,모집 완료 시 마감
2,구인중,효성렌트카 주식회사,구로구 구로동,렌트카 배차 및 회차 직원,~70세,남,월245만원,07:00~19:00,2명,"<효성렌트카 주식회사>렌트카 배차 및 회차업무 사원 2명차량운전 및 관리, 차량배차...",*상담사 알선 02-6220-8640,NaN,None,민간,구로구 구로동 렌트카 배차 및 회차 직원 모집,주6일,무관,모집 완료 시 마감
3,구인중,강남시니어클럽,서울지역,카드배송원,만 60세 이상(만 65세 이상 우대),무관,월50만원 이상,09:00~15:00,00명,업무내용: 도보를 이용하여 소화물(카드 등)을 배달하는 업무.* 참여조건1. 만 6...,02-547-8866 서울강남시니어클럽,NaN,None,민간,서울지역 카드배송원 모집,주5일,무관,모집 완료 시 마감
4,구인중,제이앤비이에스,강남구 역삼동/삼성동/논현동,가정방문 도시가스 점검원,~60세,여,월220만원,None,2명,"<제이앤비이에스>가정방문 도시가스 점검원 2명강남구 역삼동, 삼성, 논현 인근 가정...",*상담사 알선 02-6220-8640,NaN,None,민간,강남구 역삼동/삼성동/논현동 가정방문 도시가스 점검원 모집,주5일,무관,모집 완료 시 마감


In [34]:
df.columns

Index(['구인상태', '사업장명', '근무지역', '직종', '연령', '성별', '급여', '근무시간', '채용인원', '근무내용',
       '지원방법', '지원서류', '업종', '종류', '구인제목', '근무형태', '경력조건', '접수마감'],
      dtype='object')

In [35]:
df2.columns

Index(['Unnamed: 0', '구인제목', '사업장명', '업종', '사업내용', '근로자수', '주소', '모집직종',
       '모집인원', '학력', '고용형태', '근무예정지', '직무내용', '경력조건', '필요경력', '급여', '평일 근무시간',
       '근무형태', '접수마감', '전형방법', '접수방법'],
      dtype='object')

In [36]:
df2.drop(['Unnamed: 0', '사업내용', '근로자수', '주소', '학력', '고용형태'], axis=1, inplace=True)

df2.rename(columns={
    '모집직종': '직종',
    '모집인원': '채용인원',
    '근무예정지': '근무지역',
    '직무내용': '근무내용',
    '평일 근무시간': '근무시간',
    '접수방법': '지원방법',
}, inplace=True)

In [37]:
df2.columns

Index(['구인제목', '사업장명', '업종', '직종', '채용인원', '근무지역', '근무내용', '경력조건', '필요경력',
       '급여', '근무시간', '근무형태', '접수마감', '전형방법', '지원방법'],
      dtype='object')

In [38]:
df2.head()

,구인제목,사업장명,업종,직종,채용인원,근무지역,근무내용,경력조건,필요경력,급여,근무시간,근무형태,접수마감,전형방법,지원방법
0,강남구립 역삼노인요양센터 직원(사회복지사) 채용 공고,구립 역삼노인복지센터 부설 역삼노인요양센터,사회복지,사회복지사,1명,서울특별시 강남구 도곡로27길 27 (역삼동) 역삼노인요양센터,"이용 어르신 관리 및 상담, 장기요양수가 청구, 프로그램 진행 등 장기요양시설 사회...",무관,NaN,(월급) 220만원,09:00~18:00,주간,2024.05.07,"서류 ,면접","온라인,방문,우편,팩스,기타( 이메일 지원 yssenior@hanmail.net )"
1,[제2024-23호] 시립은평의마을 직원 채용 공고(계약직 생활복지사)(’24.5....,서울특별시립은평의마을,사회복지,사회복지사(사회복지시설),1명,서울특별시 은평구 갈현로15길 27 -1 (구산동) 은평의마을,"프로그램, 상담, 사무업무 등\n\r\n*근무시간/스케줄에 따라 4교대\r\nA조-...",무관,NaN,(월급) 234만원,채용공고문 참조,3교대,2024.05.16,"서류 ,면접 ,신체검사",기타( 채용공고문 확인 후 우편 또는 이메일 접수((hjpark119@bizmeka...
2,연장반(영아) 교사 모집,국공립 동화어린이집,사회복지,보육 교사,1명,서울특별시 중구 청구로 64 (신당동) 106동 1층 동화어린이집,안녕하십니까! 국공립 동화어린이집입니다. 본 어린이집에서는 영유아를 존중하고 사랑하...,무관,NaN,(월급) 106만원\r\n\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t...,15:00~19:30,주간,채용시까지(2개월이내),"서류 ,면접","전자우편,기타( 이메일 donghwa2234@naver.com )"
3,[원효로3가] 영아반 연장보육 전담교사 모집,원효어린이집,사회복지,보육 교사,1명,서울특별시 용산구 원효로41가길 14 -19 (원효로3가) (원효로3가),[영아반 연장보육 전담교사 모집 ]\n\r\n- 직무 : 야간연장 교사와 함께 영아...,무관,NaN,(월급) 106만원,15:00~19:30,주간,채용시까지(2개월이내),"서류 ,면접",전자우편
4,(재공고) 서대문구 서울형 키즈카페 북가좌1동점 종사자 채용 공고,가좌제일교회,사회복지,기타 사회복지 종사원,1명,서울특별시 서대문구 수색로8길 48 -4 (북가좌동) 3층 키즈카페,서울형 키즈카페 북가좌1동점 운영요원 1명 채용 \r\n- 시설 예산 집행 및 이용...,무관,NaN,(최소연봉) 2500만원,9:00~18:00,주간,2024.05.14,"서류 ,면접",기타( 이메일 접수 (gajwawebmaster@gmail.com) )


In [39]:
df.to_csv('민간일자리정보_정리.csv', index=False)
df2.to_csv('포털일자리정보_정리.csv', index=False)

In [43]:
len(df), len(df2)

(2006, 142)

In [41]:
result_df = pd.concat([df, df2], axis=0, ignore_index=True, join='outer')
len(result_df)

2148

In [42]:
result_df.to_csv('일자리정보_정리.csv', index=False)

1000건으로 데이터 끊기

In [54]:
public_jobs = df[df['종류'] == '공공']
private_jobs = df[df['종류'] == '민간']
needed_private_jobs = 858 - len(public_jobs)
selected_private_jobs = private_jobs.head(needed_private_jobs)
df_reduced = pd.concat([public_jobs, selected_private_jobs], ignore_index=True)
len(df_reduced)

858

In [55]:
result_df_small = pd.concat([df_reduced, df2], axis=0, ignore_index=True, join='outer')
len(result_df_small)

1000

In [56]:
result_df_small.to_csv('일자리정보_정리_1000건.csv', index=False)

In [57]:
result_df_small.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 20 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   구인상태    858 non-null    object
 1   사업장명    1000 non-null   object
 2   근무지역    1000 non-null   object
 3   직종      1000 non-null   object
 4   연령      858 non-null    object
 5   성별      858 non-null    object
 6   급여      1000 non-null   object
 7   근무시간    457 non-null    object
 8   채용인원    1000 non-null   object
 9   근무내용    1000 non-null   object
 10  지원방법    1000 non-null   object
 11  지원서류    45 non-null     object
 12  업종      142 non-null    object
 13  종류      858 non-null    object
 14  구인제목    1000 non-null   object
 15  근무형태    776 non-null    object
 16  경력조건    1000 non-null   object
 17  접수마감    1000 non-null   object
 18  필요경력    27 non-null     object
 19  전형방법    138 non-null    object
dtypes: object(20)
memory usage: 156.4+ KB
